In [ ]:
!pip3 install confluent-kafka
!pip3 install pandas
!pip3 install s3fs
!pip install boto3

In [ ]:
import pandas as pd
from confluent_kafka import Consumer
from time import sleep
import json
import socket
from s3fs import S3FileSystem
import boto3
from botocore.exceptions import NoCredentialsError
import os

In [ ]:
os.environ['AWS_ACCESS_KEY_ID'] = '<insert your access key>'
os.environ['AWS_SECRET_ACCESS_KEY'] = '<insert your secret key ;P>'
os.environ['AWS_DEFAULT_REGION'] = '<insert your regions ex: us-east-1>'

In [ ]:
conf = {
    # make sure you replace these with your broker addresses
    'bootstrap.servers': 'b-3.testmsk.sqb9zi.c10.kafka.us-east-1.amazonaws.com:9092,b-1.testmsk.sqb9zi.c10.kafka.us-east-1.amazonaws.com:9092,b-2.testmsk.sqb9zi.c10.kafka.us-east-1.amazonaws.com:9092',
    'client.id': socket.gethostname(),
     'group.id': 'foo'
}

consumer = Consumer(conf)

In [ ]:
running = True

# Initialize the S3 client
s3 = boto3.client('s3')

def basic_consume_loop(consumer, topics):
    count = 0  # Initialize a counter for file names
    bucket_name = 'data-from-kafka' # PUT YOUR BUCKET NAME HERE

    try:
        consumer.subscribe(topics)

        while running:
            msg = consumer.poll(timeout=1.0)
           
            if msg is None:
                continue
            
            if msg.error():
                print(f"Consumer error: {msg.error()}")
                continue

            # Decode the message and print it
            message_value = msg.value().decode('utf-8')
            print(message_value)

            # Write the message to an S3 bucket
            file_name = "stock_market_{}.json".format(count)
            try:
                s3.put_object(Bucket=bucket_name, Key=file_name, Body=json.dumps(message_value))
                print(f"Saved message to S3 as {file_name}")
            except NoCredentialsError:
                print("Credentials not available for accessing S3")

            count += 1

    finally:
        # Close down consumer to commit final offsets.
        consumer.close()

def shutdown():
    global running
    running = False

In [ ]:
basic_consume_loop(consumer, ["Mitch_Topic"])